In [79]:
import json
import numpy as np
import pandas as pd
from matplotlib import cm
from matplotlib import colors
    
# Loading specific tab of file
file = 'PNAD_Continua_2019_Rendimento_de_Todas_as_Fontes.xlsx'
tab = 'Efetivo de todos os trabalhos'
header_row = 13
color_list = ['#17becf', '#bcbd22', '#7f7f7f', '#e377c2', '#8c564b',
              '#9467bd', '#d62728', '#2ca02c', '#ff7f0e', '#1f77b4']
states_uf_map = {'Acre':'AC', 'Alagoas':'AL', 'Amapá':'AP', 
             'Amazonas':'AM', 'Bahia':'BA', 'Ceará':'CE', 
             'Distrito Federal':'DF', 'Espirito Santo':'ES', 'Espírito Santo':'ES', 
             'Goiás':'GO', 'Maranhão':'MA', 'Mato Grosso':'MT', 
             'Mato Grosso do Sul':'MS', 'Minas Gerais':'MG', 
             'Pará': 'PA', 'Paraíba':'PB', 'Paraná':'PR', 
             'Pernambuco':'PE', 'Piauí':'PI', 'Rio de Janeiro':'RJ', 
             'Rio Grande do Norte':'RN', 'Rio Grande do Sul':'RS', 
             'Rondônia':'RO', 'Roraima':'RR', 'Santa Catarina':'SC', 
             'São Paulo':'SP', 'Sergipe':'SE', 'Tocantins':'TO'}
all_data = pd.read_excel(file, tab, header_row)

# Extracting desired indicators and columns 
cols = list(all_data.columns)
from_index = cols.index(2012)
to_index = cols.index(2019) + 1
year_cols = (cols[from_index:to_index]) ##From 2012 forward
selec_cols = ['Nível territorial', 'Abertura geográfica'] + year_cols
df = all_data.loc[
    (all_data['ind']=='Rendimento médio mensal real da população residente, com rendimento, a preços médios do ano') &
    (all_data['sub.classe']=='Todos os trabalhos¹') &
    (all_data['Unid']=='(R$)')
][selec_cols]

## Generating dictionary with summarized data
data_map = {}
color_map = {}
data_map['years'] = year_cols
data_map['last_year'] = year_cols[-1]
data_map['last_year_index'] = len(year_cols) - 1
data_map['country_data'] = []
data_map['regions_data'] = []
data_map['states_data'] = []

last_year_map = {} 
last_year_map['country_last_year_data'] = []
last_year_map['regions_last_year_data'] = []
last_year_map['states_last_year_data'] = []

def getList(df):
    return [{'year':year,'value':float(df[year].to_numpy())
            } for year in data.columns]

def getLasValue(df):
    return float(df[df.columns[-1]].to_numpy())

def appendData(years_list, last_year_list, data, ag):
    data_map[years_list].append({'name': ag,'value': getList(data)})
    last_year_map[last_year_list].append({'name': ag,'value': getLasValue(data)})
    color_map[ag] = color_list[len(color_map) % 10]
    
for nt in df['Nível territorial'].unique():
    temp = df.loc[df['Nível territorial']==nt]
    for ag in temp['Abertura geográfica'].unique():
        data = df.loc[df['Abertura geográfica']==ag, year_cols]
        if(nt == 'País'):
            appendData('country_data', 'country_last_year_data', data, ag)
        elif(nt == 'Grandes Regiões'):
            appendData('regions_data', 'regions_last_year_data', data, ag)
        else:
            appendData('states_data', 'states_last_year_data', data, ag)
            
data_map['color_map'] = color_map

## Capturing states UF
min_value = 5000
max_value = 0
for state in last_year_map['states_last_year_data']:
    state['uf'] = states_uf_map[state['name']]
    min_value = min(min_value, state['value'])
    max_value = max(max_value, state['value'])

## Generating HTML color scale
reds = cm.get_cmap('Greys')
for state in last_year_map['states_last_year_data']:
    value = state['value']
    color_value = (value - min_value)/(max_value - min_value)
    state['color'] = colors.to_hex(reds(color_value))
    
last_year_map['last_year'] = year_cols[-1]

## Save summarized data as a json file
with open('data.json', 'w') as fp:
    json.dump(data_map, fp)
    
## Save summarized data as a json file
with open('last_year_data.json', 'w') as fp:
    json.dump(last_year_map, fp)